In [31]:
import pandas as pd

##### **Fonctions utiles pour mon script**

In [32]:
# Fonction de classification
def classification(score):
    if score in range(1, 7):
        return 'detracteur'
    elif score in range(7, 9):
        return 'neutre'
    else:
        return 'promoteur'

In [33]:
# Fonction de calcul du NPS
def compute_nps(data):
    repartition = data.value_counts(normalize=True)
    classes = ['detracteur', 'promoteur']
    for classe in classes:
        repartition[classe] = 0 if classe not in repartition.index else repartition[classe]
    return round(100*(repartition['promoteur'] - repartition['detracteur']), 2)


In [34]:
def print_what_happened(data, output_file):
    # Create a copy of the DataFrame to avoid the SettingWithCopyWarning
    data_copy = data.copy()
    
    # Replace values in the 'region' column
    data_copy['region'].replace({'Yaoundé' : 'CM Centre', 'Douala' : 'CM Littoral'}, inplace=True)
         
    # Supprimer le "CM" devant chaque région
    data_copy['region'] = data_copy['region'].str.replace('CM ', '', regex=False)
    # Specify the indicators to compute mean for
    indicators = ['qualite_couverture', 'qualite_appels', 'qualite_data_principal', 'qualite_service_client']
    
    # Group by 'region' column and aggregate
    nps_regional = data_copy.groupby('region').agg({'classe': [compute_nps, 'count'],
                                                    **{ind: 'mean' for ind in indicators}})
    
    # Reset the index to move 'region' from index to a column
    nps_regional.reset_index(inplace=True)

    # Rename columns for clarity
    nps_regional.columns = ['region', 'nps', 'nombre répondants'] + [f'{ind}_moyenne' for ind in indicators]
    
      
    # Convert DataFrame to JSON
    json_data = nps_regional.to_json(orient='records')
    
    # Write JSON to a file
    with open(output_file, 'w') as f:
        f.write(json_data)

    print("Les informations ont été enregistrées dans le fichier", output_file)

    
    # Print the result
    print("Notes Régionalisées:")
    print(nps_regional)

##### **Chargement des données**

In [35]:
data_complete = pd.read_excel('data/data_map_processed.xlsx')

In [36]:
# Filtrer les données pour ne conserver que celles concernant l'opérateur Orange
data = data_complete[data_complete['operateur_telco'] == 'Orange'].copy()


In [37]:
len(data)

1095

In [38]:
# Pour classer les scores de NPS
data.loc[:, 'classe'] = data['note_nps'].apply(classification)

In [39]:
# Calculer le NPS global
nps_global = compute_nps(data['classe'])
print("NPS Global:", nps_global)

NPS Global: -11.96


In [40]:
# Call the function to print regional scores for both NPS and quality of coverage
print_what_happened(data, "output/NPS_MARS.json")

Les informations ont été enregistrées dans le fichier output/NPS_MARS.json
Notes Régionalisées:
         region    nps  nombre répondants  qualite_couverture_moyenne  \
0      Adamaoua -22.22                 63                    6.873016   
1        Centre -25.27                277                    6.584838   
2           Est -30.77                 52                    5.961538   
3  Extrême-Nord   3.16                158                    7.670886   
4      Littoral -19.50                241                    6.506224   
5          Nord  10.48                105                    7.371429   
6    Nord-Ouest  19.35                 31                    7.064516   
7         Ouest   0.00                 95                    7.189474   
8           Sud -26.19                 42                    6.523810   
9     Sud-Ouest  16.13                 31                    6.741935   

   qualite_appels_moyenne  qualite_data_principal_moyenne  \
0                6.682540              